In [3]:
import numpy as np
import math
import time
import tensorflow as tf
import matplotlib.pyplot as plt
from icecream import ic

In [25]:
def weighted_mean_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    variance = variance * (len(values)) / (len(values) -1) 
    if len(values) == 1:
        variance = 0
    return (average, math.sqrt(variance))

def weighted_mean_and_std_o(values, weights):
    """
    Return the weighted average and standard deviation.
    """
    sum_of_weights = np.sum(weights)
    weighted_mean = np.sum(values * weights) / sum_of_weights
    n = len(weights)
    numerator = np.sum(n * weights * (values - weighted_mean)**2)
    denominator = sum_of_weights * (n - 1)

    if denominator == 0:
        return weighted_mean, 0
    else:
        return weighted_mean, math.sqrt(numerator / denominator)

In [26]:
values = np.array([1])
weights = np.array([1])
print(weighted_mean_and_std(values, weights))
print(weighted_mean_and_std_o(values, weights))

(1.0, 0.0)
(1.0, 0)


C:\Users\jtalo\AppData\Local\Temp/ipykernel_20100/1648419039.py:9: RuntimeWarning: invalid value encountered in double_scalars
  variance = variance * (len(values)) / (len(values) -1)


In [27]:
start = time.time()
for i in range(100000):
    weighted_mean_and_std(values, weights)
print("math time:", time.time() - start)
for i in range(100000):
    weighted_mean_and_std_o(values, weights)
print("numpy time:", time.time() - start)


C:\Users\jtalo\AppData\Local\Temp/ipykernel_20100/1648419039.py:9: RuntimeWarning: invalid value encountered in double_scalars
  variance = variance * (len(values)) / (len(values) -1)


math time: 3.465808629989624
numpy time: 5.065503120422363


In [137]:
def f(x):
    return tf.sin(x)

def g(x):
    return tf.cos(x)

@tf.custom_gradient
def revnet(x):
    y1 = x[0] + f(x[1])
    y2 = x[1] + g(y1)

    def grad(dy):
        with tf.GradientTape() as g_tape, tf.GradientTape() as f_tape, tf.GradientTape() as g_tape2, tf.GradientTape() as f_tape2:
            z1 = y1
            g_tape.watch(z1)
            x2 = y2 - g(z1)
            f_tape.watch(x2)
            x1 = z1 - f(x2)
            grad_g = g_tape.gradient(x2, z1)
            grad_f = f_tape.gradient(x1, x2)
        
            z1bar = dy[0] + grad_g * dy[1]
            x2bar = dy[1] + grad_f * z1bar

        return tf.concat([x1, x2, z1bar, x2bar], axis=0)

    return tf.concat([y1, y2], axis=0), grad


In [138]:
x = tf.Variable([1,0.5])


with tf.GradientTape() as tape:
    z = revnet(x)

dz = tape.gradient(z, x)

ic(z)
ic(dz)




ic| z: <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1.4794255, 0.5912437], dtype=float32)>
ic| dz: <tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1.       ,  0.5      ,  1.9958286, -0.7515044], dtype=float32)>


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1.       ,  0.5      ,  1.9958286, -0.7515044], dtype=float32)>

In [ ]:
with tf.GradientTape() as tape:
    z = g(x)

dz = tape.gradient(z, x)

ic(z)
ic(dz)

ic| z: <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.5403023 , 0.87758255], dtype=float32)>
ic| dz: <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.841471  , -0.47942555], dtype=float32)>


<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.841471  , -0.47942555], dtype=float32)>

In [ ]:
@tf.custom_gradient
def clip_gradients(y):
  def backward(dy):
    return tf.clip_by_norm(dy, 0.5)
  return y, backward


v = tf.Variable(2.0)
with tf.GradientTape() as t:
  output = clip_gradients(v * v)
print(t.gradient(output, v))  # calls "backward", which clips 4 to 2


tf.Tensor(2.0, shape=(), dtype=float32)


$2-2cos^2-sin^2 = 2-cos^2-cos^2-sin^2=2-cos^2-(cos^2+sin^2)=2-cos^2-1=1-cos^2$